# Pipeline de Series Temporales: Carga, Transformación y Feature Engineering

Este notebook encapsula el proceso de carga de datos, transformación a series temporales y creación de características y target en funciones reutilizables para forecasting.

In [13]:
# 1. Importar librerías necesarias
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from pathlib import Path
from dateutil.easter import easter
import os

In [14]:
# Importación robusta de paths centralizados
import pandas as pd
from pathlib import Path

# Definir función para obtener paths de manera robusta
def get_robust_paths():
    """Obtiene paths centralizados de manera robusta, con fallback a construcción manual si falla la importación"""
    try:
        from src.paths import (RAW_BQ_PARQUET, VALIDATED_RANGE_FECHA_FAMILIA, 
                             TS_DF_BOLLERIA_SEMANAL, TS_DF_BOLLERIA_BASELINE)
    except ImportError:
        import sys
        sys.path.append('..')
        try:
            from src.paths import (RAW_BQ_PARQUET, VALIDATED_RANGE_FECHA_FAMILIA, 
                                 TS_DF_BOLLERIA_SEMANAL, TS_DF_BOLLERIA_BASELINE)
        except ImportError:
            # Fallback: Construir paths manualmente
            print("AVISO: Usando rutas alternativas para paths centralizados")
            data_dir = Path('..') / 'data'
            RAW_BQ_PARQUET = data_dir / 'raw' / 'raw_data_bq_forecasting_20250630.parquet'
            VALIDATED_RANGE_FECHA_FAMILIA = data_dir / 'interim' / 'validated_range_fecha_familia_20250630.parquet'
            TS_DF_BOLLERIA_SEMANAL = data_dir / 'processed' / 'df_bolleria_semanal.parquet'
            TS_DF_BOLLERIA_BASELINE = data_dir / 'processed' / 'ts_df_bolleria_baseline.parquet'
    
    return {
        'RAW_BQ_PARQUET': RAW_BQ_PARQUET,
        'VALIDATED_RANGE_FECHA_FAMILIA': VALIDATED_RANGE_FECHA_FAMILIA,
        'TS_DF_BOLLERIA_SEMANAL': TS_DF_BOLLERIA_SEMANAL,
        'TS_DF_BOLLERIA_BASELINE': TS_DF_BOLLERIA_BASELINE
    }

# Obtener los paths
PATHS = get_robust_paths()
print("Paths disponibles:")
for name, path in PATHS.items():
    print(f"- {name}: {path}")

Paths disponibles:
- RAW_BQ_PARQUET: C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250630.parquet
- VALIDATED_RANGE_FECHA_FAMILIA: C:\Workspace\mlops_fleca_project\data\interim\validated_range_fecha_familia_20250630.parquet
- TS_DF_BOLLERIA_SEMANAL: C:\Workspace\mlops_fleca_project\data\processed\ts_df_bolleria_semanal.parquet
- TS_DF_BOLLERIA_BASELINE: C:\Workspace\mlops_fleca_project\data\processed\ts_df_bolleria_baseline.parquet


In [15]:
# 2. Función para cargar datos raw
def cargar_datos_raw(parquet_file):
    """
    Carga un archivo parquet, convierte columnas 'dbdate' a string si existen y devuelve un DataFrame limpio.
    
    Parámetros:
    - parquet_file: Ruta al archivo parquet (str o Path), preferiblemente desde paths centralizados
    
    Retorna:
    - DataFrame con los datos cargados
    """
    print(f"Cargando datos desde: {parquet_file}")
    table = pq.read_table(parquet_file)
    schema = table.schema
    dbdate_cols = [field.name for field in schema if str(field.type) == 'dbdate']
    for col in dbdate_cols:
        table = table.set_column(table.schema.get_field_index(col), col, table.column(col).cast('string'))
    df = table.to_pandas(ignore_metadata=True)
    return df

In [16]:
# 3. Función para transformar datos a series temporales semanales completas
def transformar_a_series_temporales(df_raw, fecha_inicio='2023-01-02', fecha_fin='2025-06-29', familia='BOLLERIA', output_path=None):
    """
    Limpia, homogeneiza y agrega los datos diarios a series semanales completas para la familia indicada.
    
    Parámetros:
    - df_raw: DataFrame con datos crudos
    - fecha_inicio: Fecha de inicio (str o datetime)
    - fecha_fin: Fecha fin (str o datetime)
    - familia: Familia de productos a filtrar (str)
    - output_path: Path opcional para guardar el resultado (str o Path)
    
    Retorna:
    - DataFrame con series temporales semanales
    """
    df = df_raw.copy()
    df['fecha'] = pd.to_datetime(df['fecha'])
    # Filtrar rango de fechas
    df = df[(df['fecha'] >= fecha_inicio) & (df['fecha'] <= fecha_fin)]
    # Homogeneizar familia si es necesario (ejemplo: 'BEBIDA' a 'BEBIDAS')
    if 'familia' in df.columns:
        df.loc[df['familia'] == 'BEBIDA', 'familia'] = 'BEBIDAS'
    # Imputar valores nulos básicos
    for col in ['base_imponible', 'total']:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    # Asegurar columnas exógenas
    if 'is_summer_peak' not in df.columns:
        df['is_summer_peak'] = 0
    if 'is_easter' not in df.columns:
        df['is_easter'] = 0
    # Calcular semana ISO
    iso = df['fecha'].dt.isocalendar()
    df['year_iso'] = iso['year']
    df['week_iso'] = iso['week']
    # Contar días por semana/familia
    conteo_dias = df.groupby(['year_iso','week_iso','familia'])['fecha'].count().reset_index(name='dias_semana')
    # Agregación semanal
    df_semanal = (
        df.groupby(['year_iso','week_iso','familia'], as_index=False)
          .agg({
             'base_imponible': 'sum',
             'is_summer_peak': 'max',
             'is_easter':      'max'
          })
        .merge(conteo_dias, on=['year_iso','week_iso','familia'])
    )
    # Filtrar solo semanas completas
    df_semanal = df_semanal[df_semanal['dias_semana'] == 7]
    # Filtrar familia
    df_bolleria_semanal = df_semanal.query(f"familia=='{familia}'").copy()
    df_bolleria_semanal.rename(columns={'year_iso':'year','week_iso':'week'}, inplace=True)
    df_bolleria_semanal = df_bolleria_semanal.sort_values(['year','week']).reset_index(drop=True)
    
    # Guardar el resultado si se proporciona un path de salida
    if output_path:
        df_bolleria_semanal.to_parquet(str(output_path), index=False)
        print(f"Series temporales guardadas en: {output_path}")
        
    return df_bolleria_semanal

In [17]:
# 4. Función para transformar datos en características y target
def generar_features_y_target(df_semanal, lags=[1,2,3,4,12,24,52], output_path=None):
    """
    Genera variables lag y variables contextuales, y separa features y target para modelado.
    
    Parámetros:
    - df_semanal: DataFrame con series temporales semanales
    - lags: Lista de lags a generar
    - output_path: Path opcional para guardar el DataFrame con features (str o Path)
    
    Retorna:
    - X: DataFrame con features
    - y: Series con target
    - df_features: DataFrame completo con features y target
    """
    df = df_semanal.copy()
    df = df.sort_values(['year','week']).reset_index(drop=True)
    # Generar lags
    for lag in lags:
        df[f'lag_{lag}'] = df['base_imponible'].shift(lag)
    # Eliminar filas con NaN (por los lags)
    df_features = df.dropna().reset_index(drop=True)
    # Selección de features y target
    feature_cols = [f'lag_{lag}' for lag in lags]
    if 'is_summer_peak' in df.columns:
        feature_cols.append('is_summer_peak')
    if 'is_easter' in df.columns:
        feature_cols.append('is_easter')
    X = df_features[feature_cols]
    y = df_features['base_imponible']
    
    # Guardar el DataFrame con features si se proporciona un path
    if output_path:
        df_features.to_parquet(str(output_path), index=False)
        print(f"DataFrame con features guardado en: {output_path}")
        
    return X, y, df_features

In [18]:
# 5. Ejemplo de uso de las funciones utilizando paths centralizados

# 1. Cargar datos raw usando path centralizado
df_raw = cargar_datos_raw(str(PATHS['RAW_BQ_PARQUET']))
print(f"Datos raw cargados: {df_raw.shape}")

# 2. Transformar a series temporales semanales completas
df_semanal = transformar_a_series_temporales(df_raw)
print(f"Datos semanales completos: {df_semanal.shape}")

# 3. Generar features y target para modelado
X, y, df_final = generar_features_y_target(df_semanal)
print(f"Features shape: {X.shape}, Target shape: {y.shape}")
print(X.head())

# 4. Guardar los resultados usando paths centralizados
df_semanal.to_parquet(str(PATHS['TS_DF_BOLLERIA_SEMANAL']), index=False)
print(f"Datos semanales guardados en: {PATHS['TS_DF_BOLLERIA_SEMANAL']}")

# Guardar el conjunto de datos con features para el modelo baseline
df_final.to_parquet(str(PATHS['TS_DF_BOLLERIA_BASELINE']), index=False)
print(f"Dataset con features guardado en: {PATHS['TS_DF_BOLLERIA_BASELINE']}")

Cargando datos desde: C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250630.parquet
Datos raw cargados: (337353, 9)
Datos semanales completos: (0, 7)
Features shape: (0, 9), Target shape: (0,)
Empty DataFrame
Columns: [lag_1, lag_2, lag_3, lag_4, lag_12, lag_24, lag_52, is_summer_peak, is_easter]
Index: []
Datos semanales guardados en: C:\Workspace\mlops_fleca_project\data\processed\ts_df_bolleria_semanal.parquet
Dataset con features guardado en: C:\Workspace\mlops_fleca_project\data\processed\ts_df_bolleria_baseline.parquet
Datos semanales completos: (0, 7)
Features shape: (0, 9), Target shape: (0,)
Empty DataFrame
Columns: [lag_1, lag_2, lag_3, lag_4, lag_12, lag_24, lag_52, is_summer_peak, is_easter]
Index: []
Datos semanales guardados en: C:\Workspace\mlops_fleca_project\data\processed\ts_df_bolleria_semanal.parquet
Dataset con features guardado en: C:\Workspace\mlops_fleca_project\data\processed\ts_df_bolleria_baseline.parquet
